In [15]:
! pip install -q pandas numpy scikit-learn xgboost


[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


# IMPORT

In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split

# LOAD DATA

In [17]:
df = pd.read_csv('../KNN_Imputation/train_imputed.csv')
df

,Employee ID,Date of Joining,Gender,Company Type,WFH Setup Available,Designation,Resource Allocation,Mental Fatigue Score,Burn Rate
0,0,2008-09-30,Female,Service,No,2,3,3.80,0.16
1,1,2008-11-30,Male,Service,Yes,1,2,5.00,0.36
2,2,2008-03-10,Female,Product,Yes,2,4,5.80,0.49
3,3,2008-11-03,Male,Service,Yes,1,1,2.60,0.20
4,4,2008-07-24,Female,Service,No,3,7,6.90,0.52
...,...,...,...,...,...,...,...,...,...
10103,10103,2008-07-04,Female,Product,No,3,6,5.90,0.51
10104,10104,2008-02-29,Male,Service,No,3,6,6.30,0.62
10105,10105,2008-01-25,Male,Service,No,3,5,5.70,0.47
10106,10106,2008-12-26,Male,Service,Yes,2,3,5.50,0.40


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10108 entries, 0 to 10107
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Employee ID           10108 non-null  int64  
 1   Date of Joining       10108 non-null  object 
 2   Gender                10108 non-null  object 
 3   Company Type          10108 non-null  object 
 4   WFH Setup Available   10108 non-null  object 
 5   Designation           10108 non-null  int64  
 6   Resource Allocation   10108 non-null  int64  
 7   Mental Fatigue Score  10108 non-null  float64
 8   Burn Rate             9624 non-null   float64
dtypes: float64(2), int64(3), object(4)
memory usage: 710.8+ KB


# EDA & Preprocessing

In [19]:
df.isnull().sum()

Employee ID               0
Date of Joining           0
Gender                    0
Company Type              0
WFH Setup Available       0
Designation               0
Resource Allocation       0
Mental Fatigue Score      0
Burn Rate               484
dtype: int64

In [20]:
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9624 entries, 0 to 10107
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Employee ID           9624 non-null   int64  
 1   Date of Joining       9624 non-null   object 
 2   Gender                9624 non-null   object 
 3   Company Type          9624 non-null   object 
 4   WFH Setup Available   9624 non-null   object 
 5   Designation           9624 non-null   int64  
 6   Resource Allocation   9624 non-null   int64  
 7   Mental Fatigue Score  9624 non-null   float64
 8   Burn Rate             9624 non-null   float64
dtypes: float64(2), int64(3), object(4)
memory usage: 751.9+ KB


In [21]:
# datetime
df['Date of Joining'] = pd.to_datetime(df['Date of Joining'])

reference_date = pd.to_datetime(pd.read_csv('../train.csv')['Date of Joining']).min()
df['Days_with_company'] = (df['Date of Joining'] - reference_date).dt.days

C:\Users\tseng\AppData\Local\Temp\ipykernel_89208\3978805335.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date of Joining'] = pd.to_datetime(df['Date of Joining'])
C:\Users\tseng\AppData\Local\Temp\ipykernel_89208\3978805335.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Days_with_company'] = (df['Date of Joining'] - reference_date).dt.days


In [22]:
df_encoded = pd.get_dummies(df, columns=['Gender', 'Company Type', 'WFH Setup Available'], drop_first=True)

df_X = df_encoded[[
        'Designation', 'Resource Allocation',
        'Mental Fatigue Score', 'Days_with_company', 'Gender_Male',
        'Company Type_Service', 'WFH Setup Available_Yes'
    ]]
df_y = df_encoded['Burn Rate']

In [23]:
df_X

,Designation,Resource Allocation,Mental Fatigue Score,Days_with_company,Gender_Male,Company Type_Service,WFH Setup Available_Yes
0,2,3,3.80,273,False,True,False
1,1,2,5.00,334,True,True,True
2,2,4,5.80,69,False,False,True
3,1,1,2.60,307,True,True,True
4,3,7,6.90,205,False,True,False
...,...,...,...,...,...,...,...
10103,3,6,5.90,185,False,False,False
10104,3,6,6.30,59,True,True,False
10105,3,5,5.70,24,True,True,False
10106,2,3,5.50,360,True,True,True


In [24]:
df_y

0        0.16
1        0.36
2        0.49
3        0.20
4        0.52
         ... 
10103    0.51
10104    0.62
10105    0.47
10106    0.40
10107    0.74
Name: Burn Rate, Length: 9624, dtype: float64

# Split train test

In [25]:
X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.2, random_state=42)

# XGBoost

In [26]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

In [27]:
# param_grid_xgb = {
#     'learning_rate': [0.1, 0.05],
#     'n_estimators': [400, 600, 800],
#     'max_depth': [3, 5, 7, 9, 12],  
#     'gamma': [0, 0.1],      
#     'subsample': [0.8, 1.0],
#     'colsample_bytree': [0.8, 1.0],
#     'objective': ['reg:absoluteerror', 'reg:squarederror']
# }

param_grid_xgb = {
    # 'booster': ['gbtree', 'gblinear'],
    'learning_rate': [0.01, 0.001],
    'n_estimators': [800, 1000, 1200],
    'max_depth': [5],  
    # 'gamma': [0, 0.1],      
    'subsample': [0.8, 1.0],
    # 'colsample_bytree': [0.8, 1.0],
    'objective': ['reg:absoluteerror']
}

In [28]:
xgb_model = xgb.XGBRegressor(
    random_state = 42, 
    n_job = -3
)

grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid_xgb,
    cv=5,
    scoring='neg_mean_absolute_error', 
    n_jobs=-3, 
    verbose=2 
)

grid_search.fit(X_train, y_train)
print(grid_search.best_params_)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


d:\IEEM_AI\HW1\.venv312\Lib\site-packages\xgboost\training.py:183: UserWarning: [00:57:22] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "n_job" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


{'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 800, 'objective': 'reg:absoluteerror', 'subsample': 0.8}


In [29]:
best_mae = -grid_search.best_score_
print(best_mae)

0.04834605487785855


In [30]:
xgb_model = grid_search.best_estimator_

In [31]:
import joblib
joblib.dump(xgb_model, '../XGBoost/final_xgb_model.joblib')
# xgb_model = joblib.load('../XGBoost/final_xgb_model.joblib')

['../XGBoost/final_xgb_model.joblib']

In [32]:
y_pred = xgb_model.predict(X_test)

In [33]:
from sklearn.metrics import mean_absolute_error, r2_score
import numpy as np

In [34]:
mae = mean_absolute_error(y_pred=y_pred, y_true=y_test)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"R-squared (R2) Score: {r2}")

Mean Absolute Error (MAE): 0.048407185533523016
R-squared (R2) Score: 0.9001049663326686


# test data

In [35]:
test_df = pd.read_csv('../KNN_Imputation/test_imputed.csv')

In [36]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1980 entries, 0 to 1979
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Employee ID           1980 non-null   int64  
 1   Date of Joining       1980 non-null   object 
 2   Gender                1980 non-null   object 
 3   Company Type          1980 non-null   object 
 4   WFH Setup Available   1980 non-null   object 
 5   Designation           1980 non-null   int64  
 6   Resource Allocation   1980 non-null   int64  
 7   Mental Fatigue Score  1980 non-null   float64
dtypes: float64(1), int64(3), object(4)
memory usage: 123.9+ KB


In [37]:
test_df['Resource Allocation'] = test_df['Resource Allocation'].fillna(df_X['Resource Allocation'].mode()[0])
test_df['Mental Fatigue Score'] = test_df['Mental Fatigue Score'].fillna(df_X['Mental Fatigue Score'].mean())
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1980 entries, 0 to 1979
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Employee ID           1980 non-null   int64  
 1   Date of Joining       1980 non-null   object 
 2   Gender                1980 non-null   object 
 3   Company Type          1980 non-null   object 
 4   WFH Setup Available   1980 non-null   object 
 5   Designation           1980 non-null   int64  
 6   Resource Allocation   1980 non-null   int64  
 7   Mental Fatigue Score  1980 non-null   float64
dtypes: float64(1), int64(3), object(4)
memory usage: 123.9+ KB


In [38]:
# datetime
test_df['Date of Joining'] = pd.to_datetime(test_df['Date of Joining'])

reference_date = pd.to_datetime(pd.read_csv('../train.csv')['Date of Joining']).min()
test_df['Days_with_company'] = (test_df['Date of Joining'] - reference_date).dt.days

In [39]:
df_encoded = pd.get_dummies(test_df, columns=['Gender', 'Company Type', 'WFH Setup Available'], drop_first=True)

df_X = df_encoded[[
        'Designation', 'Resource Allocation',
        'Mental Fatigue Score', 'Days_with_company', 'Gender_Male',
        'Company Type_Service', 'WFH Setup Available_Yes'
    ]]

In [40]:
y_pred = xgb_model.predict(df_X)

In [41]:
submission_df = pd.DataFrame({
    'Employee ID': test_df['Employee ID'],
    'Burn Rate': y_pred
})

In [42]:
submission_df

,Employee ID,Burn Rate
0,0,0.000334
1,1,0.730035
2,2,0.764636
3,3,0.407731
4,4,0.478933
...,...,...
1975,1975,0.187345
1976,1976,0.446161
1977,1977,0.357652
1978,1978,0.542091


In [43]:
submission_df.to_csv('XGBoost_rough_GridSearch_KNNimputed_v0.1.csv', index=False)